# Original model prediction

In [ ]:
import os
import sys
import csv
import time
from collections import deque
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import precision_recall_curve, auc
from data_loader import define_dataloader, load_embedding
from utils import get_performance_batchiter, print_performance, write_blackbox_output_batchiter
import data_io_tf

class Args:
    indepfile = None 
    blosum = None
    batch_size = 32
    epoch = 50
    min_epoch = 1
    early_stop = True 
    lr = 0.001 
    cuda = True 
    seed = 1039 
    mode = 'train'
    save_model = True 
    model = 'attention'
    drop_rate = 0.25        
    lin_size = 1024 
    padding = "mid"
    heads = 5
    max_len_tcr = 20 
    max_len_pep = 22
    n_fold = 5 
    idx_test_fold = 0
    idx_val_fold = -1

def train_main(trainfile_path, testfile_path, save_model_path, result_path):
    import pandas as pd
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    PRINT_EVERY_EPOCH = 1
    
    def train(model, device, train_loader, optimizer, epoch):
        model.train()
        for batch in train_loader:
            x_pep, x_tcr, y = batch.X_pep.to(device), batch.X_tcr.to(device), batch.y.to(device)
            optimizer.zero_grad()
            yhat = model(x_pep, x_tcr)
            y = y.unsqueeze(-1).expand_as(yhat)
            loss = F.binary_cross_entropy(yhat, y)
            loss.backward()
            optimizer.step()

        if epoch % PRINT_EVERY_EPOCH == 1:
            print('[TRAIN] Epoch {} Loss {:.4f}'.format(epoch, loss.item()))

    def evaluate_model(model, device, test_loader):
        model.eval()
        y_true = []
        y_scores = []
        with torch.no_grad():
            for batch in test_loader:
                x_pep, x_tcr, y = batch.X_pep.to(device), batch.X_tcr.to(device), batch.y.to(device)
                yhat = model(x_pep, x_tcr)
                y_true.extend(y.cpu().numpy())
                y_scores.extend(yhat.cpu().numpy())
        
        precision, recall, _ = precision_recall_curve(y_true, y_scores)
        auprc = auc(recall, precision)
        return auprc

    args = Args()
    device = torch.device('cuda' if args.cuda else 'cpu')
    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)
    embedding_matrix = load_embedding(args.blosum)
    trainfile = pd.read_csv(trainfile_path)
    trainfile = trainfile.reset_index().rename(columns={'index': 'idx'})
    testfile = pd.read_csv(testfile_path)
    testfile['idx'] = testfile.index + len(trainfile)
    alldata = pd.concat([trainfile, testfile], axis=0)
    idx_train = trainfile['idx'].tolist()
    idx_test = testfile['idx'].tolist()
    x_pep = alldata['Epitope'].values
    x_tcr = alldata['CDR3B'].values
    y = alldata['Affinity'].values

    train_loader = define_dataloader(x_pep[idx_train], x_tcr[idx_train], y[idx_train],
                                     args.max_len_pep, args.max_len_tcr,
                                     padding=args.padding,
                                     batch_size=args.batch_size, device=device)
    test_loader = define_dataloader(x_pep[idx_test], x_tcr[idx_test], y[idx_test],
                                    maxlen_pep=train_loader['pep_length'],
                                    maxlen_tcr=train_loader['tcr_length'],
                                    padding=args.padding,
                                    batch_size=args.batch_size, device=device)

    if args.model == 'attention':
        from attention import Net
    else:
        raise ValueError('unknown model name')

    model = Net(embedding_matrix, args).to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    if args.mode == 'train':
        t0 = time.time()
        lossArraySize = 10
        lossArray = deque([sys.maxsize], maxlen=lossArraySize)
        for epoch in range(1, args.epoch + 1):
            train(model, device, train_loader['loader'], optimizer, epoch)
            perf_test = get_performance_batchiter(test_loader['loader'], model, device)
            lossArray.append(perf_test['loss'])
            average_loss_change = sum(np.abs(np.diff(lossArray))) / lossArraySize
            auprc = evaluate_model(model, device, test_loader['loader'])
            print(f'[EPOCH {epoch}] AUPRC: {auprc:.4f}')

            if epoch > args.min_epoch and average_loss_change < 10 and args.early_stop:
                break
        
        print('[TEST ] {} ----------------'.format(epoch))
        perf_test = get_performance_batchiter(test_loader['loader'], model, device)
        print_performance(perf_test)

        if args.save_model:
            wf_open1 = open(result_path + 'probability.csv', 'w', newline='')
            wf1 = csv.writer(wf_open1, delimiter=',')
            wf1.writerow(['Epitope', 'CDR3B', 'y_true', 'y_pred', 'y_prob'])
            write_blackbox_output_batchiter(test_loader, model, wf1, device, ifscore=True)
            wf_open1.close()
                        
            torch.save(model.state_dict(), save_model_path)
name=['1']
me=['health']
for i in name:
    for j in me:
        trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/data/combined_dataset.csv"
        testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/"+i+'_'+j+'.csv'
        save_modle_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Original/"
        result_path="./result/Original/seen/three/"+i+'_'+j+ 'original.ckpt'
        train_main(trainfile_path,testfile_path,save_modle_path,result_path)

In [1]:
import os
import sys
import csv
import time
from collections import deque
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from data_loader import define_dataloader, load_embedding, load_data_split
from utils import str2bool, timeSince, get_performance_batchiter, print_performance, write_blackbox_output_batchiter
import data_io_tf


class Args:
    indepfile=None 
    blosum=None
    batch_size=32
    epoch=20
    min_epoch=1
    early_stop=True 
    lr=0.001 
    cuda=True #True 
    seed=1039 
    mode='train'
    save_model=True 
    model='attention'
    drop_rate=0.25        
    lin_size=1024 
    padding="mid"
    heads=5
    max_len_tcr=20 
    max_len_pep=22
    n_fold=5 
    idx_test_fold=0
    idx_val_fold=-1
    #split_type='tcr'

def train_main(testfile_path,modelfile_path, result_path):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score, auc, roc_curve
    import time
    import psutil    
    start_time = time.time()
    start_memory = psutil.Process().memory_info().rss

    device = torch.device('cuda')
    args = Args()

    if torch.cuda.is_available() and not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")
    device = torch.device('cuda' if args.cuda else 'cpu')

    testfile = pd.read_csv(testfile_path)
    testfile['idx'] = testfile.index
    x_pep = testfile['Epitope'].values
    x_tcr = testfile['CDR3B'].values
    y = testfile['Affinity'].values

    embedding_matrix = load_embedding(args.blosum)
    
    test_loader = define_dataloader(x_pep, x_tcr, y,
                                    maxlen_pep=args.max_len_pep,
                                    maxlen_tcr=args.max_len_tcr,
                                    padding=args.padding,
                                    batch_size=args.batch_size, device=device)
    
    if args.model == 'attention':
        from attention import Net
    else:
        raise ValueError('unknown model name')
    
    model = Net(embedding_matrix, args).to(device)
    model.load_state_dict(torch.load(modelfile_path, map_location=device))
    print('[PREDICT] ----------------')
    perf_test = get_performance_batchiter(test_loader['loader'], model, device)
    print_performance(perf_test)
    wf_open1 = open(result_path + 'probability.csv', 'w', newline='')
    wf1 = csv.writer(wf_open1, delimiter=',')
    wf1.writerow([ 'Epitope', 'CDR3B', 'y_true', 'y_pred','y_prob'])
    write_blackbox_output_batchiter(test_loader, model, wf1, device, ifscore=True)
    wf_open1.close()


In [ ]:
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
        testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/"+i+'_'+j+'.csv'
        modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Original/original.ckpt"
        result_path="./evaluate/repeat10/Prediction/all/seen/three/"+i+'_'+j
        train_main(testfile_path,modelfile_path,result_path)

name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
        testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/"+i+'_'+j+'.csv'
        modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Original/original.ckpt"
        result_path="./evaluate/repeat10/Prediction/all/seen/seven/"+i+'_'+j
        train_main(testfile_path,modelfile_path,result_path)

        
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
         testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/"+i+'_'+j+'.csv'
        result_path="./evaluate/repeat10/Prediction/all/unseen/"+i+'_'+j
        modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Original/original.ckpt"
        train_main(testfile_path,modelfile_path,result_path)


# Model retraining

In [2]:
import os
import sys
import csv
import time
from collections import deque
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import precision_recall_curve, auc
from data_loader import define_dataloader, load_embedding
from utils import get_performance_batchiter, print_performance, write_blackbox_output_batchiter
import data_io_tf

class Args:
    indepfile = None 
    blosum = None
    batch_size = 32
    epoch = 50
    min_epoch = 1
    early_stop = True 
    lr = 0.001 
    cuda = True 
    seed = 1039 
    mode = 'train'
    save_model = True 
    model = 'attention'
    drop_rate = 0.25        
    lin_size = 1024 
    padding = "mid"
    heads = 5
    max_len_tcr = 20 
    max_len_pep = 22
    n_fold = 5 
    idx_test_fold = 0
    idx_val_fold = -1

def train_main(trainfile_path, testfile_path, save_model_path, result_path):
    import pandas as pd
    if torch.cuda.is_available():
        print("gpu enabled")
    else:
        print("no gpu")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    PRINT_EVERY_EPOCH = 1

    def train(model, device, train_loader, optimizer, epoch):
        model.train()
        for batch in train_loader:
            x_pep, x_tcr, y = batch.X_pep.to(device), batch.X_tcr.to(device), batch.y.to(device)
            optimizer.zero_grad()
            yhat = model(x_pep, x_tcr)
            y = y.unsqueeze(-1).expand_as(yhat)
            loss = F.binary_cross_entropy(yhat, y)
            loss.backward()
            optimizer.step()

        if epoch % PRINT_EVERY_EPOCH == 1:
            print('[TRAIN] Epoch {} Loss {:.4f}'.format(epoch, loss.item()))

    def evaluate_model(model, device, test_loader):
        model.eval()
        y_true = []
        y_scores = []
        with torch.no_grad():
            for batch in test_loader:
                x_pep, x_tcr, y = batch.X_pep.to(device), batch.X_tcr.to(device), batch.y.to(device)
                yhat = model(x_pep, x_tcr)
                y_true.extend(y.cpu().numpy())
                y_scores.extend(yhat.cpu().numpy())
        
        precision, recall, _ = precision_recall_curve(y_true, y_scores)
        auprc = auc(recall, precision)
        return auprc

    args = Args()

    device = torch.device('cuda' if args.cuda else 'cpu')

    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    embedding_matrix = load_embedding(args.blosum)

    trainfile = pd.read_csv(trainfile_path)
    trainfile = trainfile.reset_index().rename(columns={'index': 'idx'})
    testfile = pd.read_csv(testfile_path)
    testfile['idx'] = testfile.index + len(trainfile)
    alldata = pd.concat([trainfile, testfile], axis=0)
    idx_train = trainfile['idx'].tolist()
    idx_test = testfile['idx'].tolist()
    x_pep = alldata['Epitope'].values
    x_tcr = alldata['CDR3B'].values
    y = alldata['Affinity'].values

    train_loader = define_dataloader(x_pep[idx_train], x_tcr[idx_train], y[idx_train],
                                     args.max_len_pep, args.max_len_tcr,
                                     padding=args.padding,
                                     batch_size=args.batch_size, device=device)
    test_loader = define_dataloader(x_pep[idx_test], x_tcr[idx_test], y[idx_test],
                                    maxlen_pep=train_loader['pep_length'],
                                    maxlen_tcr=train_loader['tcr_length'],
                                    padding=args.padding,
                                    batch_size=args.batch_size, device=device)

    if args.model == 'attention':
        from attention import Net
    else:
        raise ValueError('unknown model name')

    model = Net(embedding_matrix, args).to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    if args.mode == 'train':
        t0 = time.time()
        lossArraySize = 10
        lossArray = deque([sys.maxsize], maxlen=lossArraySize)
        for epoch in range(1, args.epoch + 1):
            train(model, device, train_loader['loader'], optimizer, epoch)
            perf_test = get_performance_batchiter(test_loader['loader'], model, device)
            lossArray.append(perf_test['loss'])
            average_loss_change = sum(np.abs(np.diff(lossArray))) / lossArraySize
            auprc = evaluate_model(model, device, test_loader['loader'])
            print(f'[EPOCH {epoch}] AUPRC: {auprc:.4f}')

            if epoch > args.min_epoch and average_loss_change < 10 and args.early_stop:
                break
        
        print('[TEST ] {} ----------------'.format(epoch))
        perf_test = get_performance_batchiter(test_loader['loader'], model, device)
        print_performance(perf_test)

        if args.save_model:
            wf_open1 = open(result_path + 'probability.csv', 'w', newline='')
            wf1 = csv.writer(wf_open1, delimiter=',')
            wf1.writerow(['Epitope', 'CDR3B', 'y_true', 'y_pred', 'y_prob'])
            write_blackbox_output_batchiter(test_loader, model, wf1, device, ifscore=True)
            wf_open1.close()
                        
            torch.save(model.state_dict(), save_model_path)


# pair50

In [ ]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+j+"train.csv"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair50/"+i+"/"+k+'_'+"1_1test.csv"
            save_modle_path="./model/Retrain/pair50/"+i+"/"+k+'_'+j+ '.ckpt'
            result_path="./result/pair50/seen/test/"+i+"/"+k+'_'+j
            train_main(trainfile_path,testfile_path,save_modle_path,result_path) 

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+j+"train.csv"
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1test.csv"
            save_modle_path="./model/Retrain/pair50/"+i+"/"+k+'_'+j+ '.ckpt'
            result_path="./result/pair50/seen/test/"+i+"/"+k+'_'+j
            train_main(trainfile_path,testfile_path,save_modle_path,result_path)     

gpu enabled


/tmp/ipykernel_44157/3796765193.py:86: DtypeWarning: Columns (5,6,7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  trainfile = pd.read_csv(trainfile_path)
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


3414 1145 1682 2877
[EPOCH 1] AUPRC: 0.7670
3784 775 1935 2624
[EPOCH 2] AUPRC: 0.7912
3675 884 1680 2879
[EPOCH 3] AUPRC: 0.8113
3701 858 1659 2900
[EPOCH 4] AUPRC: 0.8151
3424 1135 1372 3187
[EPOCH 5] AUPRC: 0.8210
3815 744 1708 2851
[EPOCH 6] AUPRC: 0.8284
3693 866 1522 3037
[EPOCH 7] AUPRC: 0.8316
3446 1113 1283 3276
[EPOCH 8] AUPRC: 0.8356
3869 690 1679 2880
[EPOCH 9] AUPRC: 0.8383
3514 1045 1358 3201
[EPOCH 10] AUPRC: 0.8344
3630 929 1403 3156
[EPOCH 11] AUPRC: 0.8437
3419 1140 1181 3378
[EPOCH 12] AUPRC: 0.8464
3677 882 1406 3153
[EPOCH 13] AUPRC: 0.8428
3405 1154 1125 3434
[EPOCH 14] AUPRC: 0.8493
3417 1142 1187 3372
[EPOCH 15] AUPRC: 0.8465
3309 1250 1104 3455
[EPOCH 16] AUPRC: 0.8438
3586 973 1328 3231
[EPOCH 17] AUPRC: 0.8435
3316 1243 1107 3452
[EPOCH 18] AUPRC: 0.8413
3326 1233 1114 3445
[EPOCH 19] AUPRC: 0.8498
3524 1035 1233 3326
[EPOCH 20] AUPRC: 0.8419
3467 1092 1192 3367
[EPOCH 21] AUPRC: 0.8469
3176 1383 984 3575
[EPOCH 22] AUPRC: 0.8447
3388 1171 1074 3485
[EPOCH 23

/tmp/ipykernel_44157/3796765193.py:86: DtypeWarning: Columns (5,6,7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  trainfile = pd.read_csv(trainfile_path)
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


3594 965 1767 2792
[EPOCH 1] AUPRC: 0.7841
3746 813 1781 2778
[EPOCH 2] AUPRC: 0.8030
4028 531 2087 2472
[EPOCH 3] AUPRC: 0.8165
3600 959 1511 3048
[EPOCH 4] AUPRC: 0.8171
3353 1206 1221 3338
[EPOCH 5] AUPRC: 0.8218
3390 1169 1243 3316
[EPOCH 6] AUPRC: 0.8311
3447 1112 1250 3309
[EPOCH 7] AUPRC: 0.8372
3504 1055 1230 3329
[EPOCH 8] AUPRC: 0.8379
3709 850 1437 3122
[EPOCH 9] AUPRC: 0.8455
3298 1261 1094 3465
[EPOCH 10] AUPRC: 0.8377
3475 1084 1187 3372
[EPOCH 11] AUPRC: 0.8447
3498 1061 1224 3335
[EPOCH 12] AUPRC: 0.8442
3482 1077 1198 3361
[EPOCH 13] AUPRC: 0.8450
3561 998 1308 3251
[EPOCH 14] AUPRC: 0.8439
3711 848 1414 3145
[EPOCH 15] AUPRC: 0.8452
3307 1252 1062 3497
[EPOCH 16] AUPRC: 0.8413
3527 1032 1261 3298
[EPOCH 17] AUPRC: 0.8401
3482 1077 1199 3360
[EPOCH 18] AUPRC: 0.8424
3531 1028 1256 3303
[EPOCH 19] AUPRC: 0.8378
3465 1094 1207 3352
[EPOCH 20] AUPRC: 0.8429
3063 1496 900 3659
[EPOCH 21] AUPRC: 0.8387
3460 1099 1224 3335
[EPOCH 22] AUPRC: 0.8389
3303 1256 1080 3479
[EPOCH 

...

4438 121 3360 1199
[EPOCH 1] AUPRC: 0.7868
4437 122 3197 1362
[EPOCH 2] AUPRC: 0.8081
4348 211 2792 1767
[EPOCH 3] AUPRC: 0.8178
4373 186 2764 1795
[EPOCH 4] AUPRC: 0.8224
4403 156 2708 1851
[EPOCH 5] AUPRC: 0.8360
4378 181 2566 1993
[EPOCH 6] AUPRC: 0.8454
4353 206 2476 2083
[EPOCH 7] AUPRC: 0.8462
4395 164 2554 2005
[EPOCH 8] AUPRC: 0.8524
4390 169 2557 2002
[EPOCH 9] AUPRC: 0.8522
4391 168 2547 2012
[EPOCH 10] AUPRC: 0.8542
4428 131 2619 1940
[EPOCH 11] AUPRC: 0.8588
4361 198 2344 2215
[EPOCH 12] AUPRC: 0.8592
4382 177 2465 2094
[EPOCH 13] AUPRC: 0.8578
4391 168 2446 2113
[EPOCH 14] AUPRC: 0.8617
4312 247 2230 2329
[EPOCH 15] AUPRC: 0.8588
4319 240 2266 2293
[EPOCH 16] AUPRC: 0.8589
4440 119 2634 1925
[EPOCH 17] AUPRC: 0.8616
4319 240 2192 2367
[EPOCH 18] AUPRC: 0.8639
4408 151 2492 2067
[EPOCH 19] AUPRC: 0.8650
4342 217 2249 2310
[EPOCH 20] AUPRC: 0.8638
4369 190 2398 2161
[EPOCH 21] AUPRC: 0.8618
4354 205 2381 2178
[EPOCH 22] AUPRC: 0.8615
4344 215 2273 2286
[EPOCH 23] AUPRC: 0.86

/tmp/ipykernel_44157/3796765193.py:86: DtypeWarning: Columns (5,6,7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  trainfile = pd.read_csv(trainfile_path)
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


4243 316 2809 1750
[EPOCH 1] AUPRC: 0.7771
4388 171 2890 1669
[EPOCH 2] AUPRC: 0.8124
4406 153 2836 1723
[EPOCH 3] AUPRC: 0.8282
4405 154 2813 1746
[EPOCH 4] AUPRC: 0.8352
4466 93 2991 1568
[EPOCH 5] AUPRC: 0.8400
4386 173 2567 1992
[EPOCH 6] AUPRC: 0.8481
4303 256 2336 2223
[EPOCH 7] AUPRC: 0.8507
4400 159 2603 1956
[EPOCH 8] AUPRC: 0.8538
4286 273 2222 2337
[EPOCH 9] AUPRC: 0.8568
4407 152 2563 1996
[EPOCH 10] AUPRC: 0.8586
4364 195 2361 2198
[EPOCH 11] AUPRC: 0.8571
4352 207 2344 2215
[EPOCH 12] AUPRC: 0.8617
4306 253 2120 2439
[EPOCH 13] AUPRC: 0.8649
4308 251 2270 2289
[EPOCH 14] AUPRC: 0.8592
4303 256 2243 2316
[EPOCH 15] AUPRC: 0.8595
4326 233 2235 2324
[EPOCH 16] AUPRC: 0.8650
4278 281 2136 2423
[EPOCH 17] AUPRC: 0.8640
4270 289 2072 2487
[EPOCH 18] AUPRC: 0.8676
4340 219 2254 2305
[EPOCH 19] AUPRC: 0.8635
4279 280 2115 2444
[EPOCH 20] AUPRC: 0.8656
4266 293 2035 2524
[EPOCH 21] AUPRC: 0.8666
4257 302 2073 2486
[EPOCH 22] AUPRC: 0.8647
4325 234 2219 2340
[EPOCH 23] AUPRC: 0.865

/tmp/ipykernel_44157/3796765193.py:86: DtypeWarning: Columns (5,6,7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  trainfile = pd.read_csv(trainfile_path)
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


4525 34 3853 706
[EPOCH 1] AUPRC: 0.7802
4525 34 3683 876
[EPOCH 2] AUPRC: 0.8108
4526 33 3620 939
[EPOCH 3] AUPRC: 0.8251
4536 23 3573 986
[EPOCH 4] AUPRC: 0.8391
4548 11 3769 790
[EPOCH 5] AUPRC: 0.8443
4539 20 3596 963
[EPOCH 6] AUPRC: 0.8499
4541 18 3509 1050
[EPOCH 7] AUPRC: 0.8540
4526 33 3336 1223
[EPOCH 8] AUPRC: 0.8550
4528 31 3432 1127
[EPOCH 9] AUPRC: 0.8435
4530 29 3281 1278
[EPOCH 10] AUPRC: 0.8569
4517 42 3157 1402
[EPOCH 11] AUPRC: 0.8548
4505 54 3054 1505
[EPOCH 12] AUPRC: 0.8568
4511 48 3102 1457
[EPOCH 13] AUPRC: 0.8543
4519 40 3290 1269
[EPOCH 14] AUPRC: 0.8474
4495 64 3089 1470
[EPOCH 15] AUPRC: 0.8508
4442 117 3050 1509
[EPOCH 16] AUPRC: 0.8389
4414 145 2837 1722
[EPOCH 17] AUPRC: 0.8418
4330 229 2646 1913
[EPOCH 18] AUPRC: 0.8383
4441 118 2815 1744
[EPOCH 19] AUPRC: 0.8499
4420 139 2690 1869
[EPOCH 20] AUPRC: 0.8535
4373 186 2877 1682
[EPOCH 21] AUPRC: 0.8306
4268 291 2519 2040
[EPOCH 22] AUPRC: 0.8304
4227 332 2362 2197
[EPOCH 23] AUPRC: 0.8393
4118 441 2109 2450

# pair300

In [15]:
pair=['more300','300','200','100','10']
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            for l in pair:
                trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/more300/"+i+"/"+k+'_'+"1_1test.csv"
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                save_modle_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair300/seen/"+l+"/"+i+"/"+k+'_'+j+ '.ckpt'
                train_main(trainfile_path,testfile_path,save_modle_path,result_path)   
                

pair=['more300','300','200','100','10']
database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1']
for i in database:
    for j in name:
        for k in name1:
            for l in pair:
                trainfile_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/"+l+"/"+i+"/"+k+'_'+j+"train.csv"
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/pair300/more300/"+i+"/"+k+'_'+"1_1test.csv"
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/result/pair300/seen/"+l+"/"+i+"/"+k+'_'+j
                save_modle_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair300/seen/"+l+"/"+i+"/"+k+'_'+j+ '.ckpt'
                train_main(trainfile_path,testfile_path,save_modle_path,result_path)   

gpu enabled


/tmp/ipykernel_32923/3796765193.py:86: DtypeWarning: Columns (5,6,7,8,10) have mixed types. Specify dtype option on import or set low_memory=False.
  trainfile = pd.read_csv(trainfile_path)
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


1161 458 566 1053
[EPOCH 1] AUPRC: 0.7614
1247 372 580 1039
[EPOCH 2] AUPRC: 0.7897
1383 236 721 898
[EPOCH 3] AUPRC: 0.8026
1402 217 693 926
[EPOCH 4] AUPRC: 0.8157
1191 428 462 1157
[EPOCH 5] AUPRC: 0.8170
1326 293 544 1075
[EPOCH 6] AUPRC: 0.8271
1261 358 490 1129
[EPOCH 7] AUPRC: 0.8262
1234 385 428 1191
[EPOCH 8] AUPRC: 0.8372
1228 391 420 1199
[EPOCH 9] AUPRC: 0.8402
1255 364 442 1177
[EPOCH 10] AUPRC: 0.8371
1352 267 553 1066
[EPOCH 11] AUPRC: 0.8425
1216 403 406 1213
[EPOCH 12] AUPRC: 0.8401
1194 425 390 1229
[EPOCH 13] AUPRC: 0.8437
1256 363 419 1200
[EPOCH 14] AUPRC: 0.8453
1195 424 374 1245
[EPOCH 15] AUPRC: 0.8498
1221 398 388 1231
[EPOCH 16] AUPRC: 0.8545
1248 371 376 1243
[EPOCH 17] AUPRC: 0.8558
1390 229 520 1099
[EPOCH 18] AUPRC: 0.8552
1235 384 384 1235
[EPOCH 19] AUPRC: 0.8530
1228 391 364 1255
[EPOCH 20] AUPRC: 0.8589
1276 343 403 1216
[EPOCH 21] AUPRC: 0.8583
1207 412 349 1270
[EPOCH 22] AUPRC: 0.8525
1234 385 380 1239
[EPOCH 23] AUPRC: 0.8558
1172 447 330 1289
[EPO

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


gpu enabled
158 1461 72 1547
[EPOCH 1] AUPRC: 0.6021
416 1203 237 1382
[EPOCH 2] AUPRC: 0.5993
688 931 461 1158
[EPOCH 3] AUPRC: 0.5758
581 1038 338 1281
[EPOCH 4] AUPRC: 0.5971
798 821 510 1109
[EPOCH 5] AUPRC: 0.6215
957 662 650 969
[EPOCH 6] AUPRC: 0.5940
610 1009 363 1256
[EPOCH 7] AUPRC: 0.5853
1251 368 1031 588
[EPOCH 8] AUPRC: 0.6139
584 1035 373 1246
[EPOCH 9] AUPRC: 0.5901
1342 277 1193 426
[EPOCH 10] AUPRC: 0.5799
879 740 594 1025
[EPOCH 11] AUPRC: 0.6104
1115 504 882 737
[EPOCH 12] AUPRC: 0.5953
399 1220 262 1357
[EPOCH 13] AUPRC: 0.5543
1293 326 1068 551
[EPOCH 14] AUPRC: 0.6038
1159 460 965 654
[EPOCH 15] AUPRC: 0.5649
535 1084 300 1319
[EPOCH 16] AUPRC: 0.5961
771 848 556 1063
[EPOCH 17] AUPRC: 0.5752
956 663 667 952
[EPOCH 18] AUPRC: 0.5995
989 630 667 952
[EPOCH 19] AUPRC: 0.6006
780 839 499 1120
[EPOCH 20] AUPRC: 0.6076
672 947 452 1167
[EPOCH 21] AUPRC: 0.5858
983 636 732 887
[EPOCH 22] AUPRC: 0.5886
1125 494 838 781
[EPOCH 23] AUPRC: 0.6027
708 911 449 1170
[EPOCH 24

...

gpu enabled
662 957 667 952
[EPOCH 1] AUPRC: 0.5112
1530 89 1501 118
[EPOCH 2] AUPRC: 0.5282
483 1136 437 1182
[EPOCH 3] AUPRC: 0.5117
929 690 750 869
[EPOCH 4] AUPRC: 0.5567
913 706 743 876
[EPOCH 5] AUPRC: 0.5799
1165 454 997 622
[EPOCH 6] AUPRC: 0.5909
746 873 502 1117
[EPOCH 7] AUPRC: 0.5961
1061 558 849 770
[EPOCH 8] AUPRC: 0.6044
925 694 677 942
[EPOCH 9] AUPRC: 0.6120
1063 556 758 861
[EPOCH 10] AUPRC: 0.6149
846 773 600 1019
[EPOCH 11] AUPRC: 0.6155
717 902 441 1178
[EPOCH 12] AUPRC: 0.6279
1139 480 848 771
[EPOCH 13] AUPRC: 0.6254
954 665 592 1027
[EPOCH 14] AUPRC: 0.6412
1003 616 694 925
[EPOCH 15] AUPRC: 0.6475
805 814 526 1093
[EPOCH 16] AUPRC: 0.6453
1086 533 732 887
[EPOCH 17] AUPRC: 0.6530
893 726 561 1058
[EPOCH 18] AUPRC: 0.6472
919 700 570 1049
[EPOCH 19] AUPRC: 0.6460
858 761 512 1107
[EPOCH 20] AUPRC: 0.6579
958 661 641 978
[EPOCH 21] AUPRC: 0.6598
944 675 597 1022
[EPOCH 22] AUPRC: 0.6533
1010 609 637 982
[EPOCH 23] AUPRC: 0.6554
916 703 547 1072
[EPOCH 24] AUPRC: 

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


gpu enabled
988 631 1004 615
[EPOCH 1] AUPRC: 0.5042
105 1514 83 1536
[EPOCH 2] AUPRC: 0.5234
1241 378 1204 415
[EPOCH 3] AUPRC: 0.5184
866 753 810 809
[EPOCH 4] AUPRC: 0.5185
813 806 799 820
[EPOCH 5] AUPRC: 0.5076
1039 580 959 660
[EPOCH 6] AUPRC: 0.5304
507 1112 443 1176
[EPOCH 7] AUPRC: 0.5377
1132 487 990 629
[EPOCH 8] AUPRC: 0.5532
475 1144 344 1275
[EPOCH 9] AUPRC: 0.5784
1149 470 949 670
[EPOCH 10] AUPRC: 0.5998
858 761 696 923
[EPOCH 11] AUPRC: 0.5688
523 1096 357 1262
[EPOCH 12] AUPRC: 0.6021
825 794 623 996
[EPOCH 13] AUPRC: 0.5801
639 980 384 1235
[EPOCH 14] AUPRC: 0.5999
1190 429 968 651
[EPOCH 15] AUPRC: 0.6013
937 682 640 979
[EPOCH 16] AUPRC: 0.6181
625 994 365 1254
[EPOCH 17] AUPRC: 0.6188
810 809 513 1106
[EPOCH 18] AUPRC: 0.6307
735 884 448 1171
[EPOCH 19] AUPRC: 0.6286
645 974 393 1226
[EPOCH 20] AUPRC: 0.6334
716 903 422 1197
[EPOCH 21] AUPRC: 0.6303
838 781 559 1060
[EPOCH 22] AUPRC: 0.6222
583 1036 295 1324
[EPOCH 23] AUPRC: 0.6448
872 747 542 1077
[EPOCH 24] AUP

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


gpu enabled
302 1317 312 1307
[EPOCH 1] AUPRC: 0.5022
528 1091 511 1108
[EPOCH 2] AUPRC: 0.5126
793 826 782 837
[EPOCH 3] AUPRC: 0.5120
433 1186 396 1223
[EPOCH 4] AUPRC: 0.5188
480 1139 421 1198
[EPOCH 5] AUPRC: 0.5222
1177 442 1134 485
[EPOCH 6] AUPRC: 0.5328
575 1044 505 1114
[EPOCH 7] AUPRC: 0.5249
1008 611 930 689
[EPOCH 8] AUPRC: 0.5354
647 972 574 1045
[EPOCH 9] AUPRC: 0.5376
1068 551 970 649
[EPOCH 10] AUPRC: 0.5403
514 1105 439 1180
[EPOCH 11] AUPRC: 0.5404
744 875 633 986
[EPOCH 12] AUPRC: 0.5544
822 797 686 933
[EPOCH 13] AUPRC: 0.5488
319 1300 255 1364
[EPOCH 14] AUPRC: 0.5431
1334 285 1255 364
[EPOCH 15] AUPRC: 0.5551
587 1032 440 1179
[EPOCH 16] AUPRC: 0.5535
866 753 720 899
[EPOCH 17] AUPRC: 0.5596
569 1050 477 1142
[EPOCH 18] AUPRC: 0.5533
890 729 743 876
[EPOCH 19] AUPRC: 0.5479
781 838 675 944
[EPOCH 20] AUPRC: 0.5513
552 1067 456 1163
[EPOCH 21] AUPRC: 0.5513
1112 507 960 659
[EPOCH 22] AUPRC: 0.5670
162 1457 123 1496
[EPOCH 23] AUPRC: 0.5608
1272 347 1129 490
[EPOCH

In [5]:
import os
import sys
import csv
import time
from collections import deque
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from data_loader import define_dataloader, load_embedding, load_data_split
from utils import str2bool, timeSince, get_performance_batchiter, print_performance, write_blackbox_output_batchiter
import data_io_tf


class Args:
    indepfile=None 
    blosum=None
    batch_size=32
    epoch=20
    min_epoch=1
    early_stop=True 
    lr=0.001 
    cuda=True #True 
    seed=1039 
    mode='train'
    save_model=True 
    model='attention'
    drop_rate=0.25        
    lin_size=1024 
    padding="mid"
    heads=5
    max_len_tcr=20 
    max_len_pep=22
    n_fold=5 
    idx_test_fold=0
    idx_val_fold=-1
    #split_type='tcr'

def train_main(testfile_path,modelfile_path, result_path):
    import pandas as pd
    from sklearn.metrics import accuracy_score, roc_auc_score, recall_score, f1_score, matthews_corrcoef, precision_score, auc, roc_curve

    device = torch.device('cuda')
    args = Args()

    if torch.cuda.is_available() and not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")
    device = torch.device('cuda' if args.cuda else 'cpu')

    testfile = pd.read_csv(testfile_path)
    print(testfile.index)
    testfile['idx'] = testfile.index
    x_pep = testfile['Epitope'].values
    x_tcr = testfile['CDR3B'].values
    y = testfile['Affinity'].values

    embedding_matrix = load_embedding(args.blosum)
    
    test_loader = define_dataloader(x_pep, x_tcr, y,
                                    maxlen_pep=args.max_len_pep,
                                    maxlen_tcr=args.max_len_tcr,
                                    padding=args.padding,
                                    batch_size=args.batch_size, device=device)
    
    if args.model == 'attention':
        from attention import Net
    else:
        raise ValueError('unknown model name')
    
    model = Net(embedding_matrix, args).to(device)
    model.load_state_dict(torch.load(modelfile_path, map_location=device))
    print('[PREDICT] ----------------')
    perf_test = get_performance_batchiter(test_loader['loader'], model, device)
    print_performance(perf_test)
    wf_open1 = open(result_path + 'probability.csv', 'w', newline='')
    wf1 = csv.writer(wf_open1, delimiter=',')
    wf1.writerow([ 'Epitope', 'CDR3B', 'y_true', 'y_pred','y_prob'])
    write_blackbox_output_batchiter(test_loader, model, wf1, device, ifscore=True)
    wf_open1.close()


# validation

In [21]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair50/"+i+"/"+k+'_'+j+'.ckpt'
            result_path="./result/pair50/seen/validation/"+i+"/"+k+'_'+j
            train_main(testfile_path,modelfile_path, result_path) 

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Validation/"+i+"/"+k+'_'+"1_1Validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair50/"+i+"/"+k+'_'+j+'.ckpt'
            result_path="./result/pair50/seen/validation/"+i+"/"+k+'_'+j
            train_main(testfile_path,modelfile_path,result_path) 
            

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
1348 1116 459 2005
accuracy   0.6804
auc        0.7820
f1macro    0.6746
f1micro    0.6804
loss       14562.6164
precision0 0.7460
precision1 0.6424
recall0    0.5471
recall1    0.8137



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
1624 840 626 1838
accuracy   0.7025
auc        0.7818
f1macro    0.7020
f1micro    0.7025
loss       12378.5144
precision0 0.7218
precision1 0.6863
recall0    0.6591
recall1    0.7459



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


...

[PREDICT] ----------------
999 1465 680 1784
accuracy   0.5647
auc        0.6455
f1macro    0.5534
f1micro    0.5647
loss       9023.6681
precision0 0.5950
precision1 0.5491
recall0    0.4054
recall1    0.7240



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
652 1812 406 2058
accuracy   0.5499
auc        0.6295
f1macro    0.5100
f1micro    0.5499
loss       16245.4047
precision0 0.6163
precision1 0.5318
recall0    0.2646
recall1    0.8352



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
394 2070 290 2174
accuracy   0.5211
auc        0.6249
f1macro    0.4492
f1micro    0.5211
loss       21485.8966
precision0 0.5760
precision1 0.5123
recall0    0.1599
recall1    0.8823



# unknown-test

In [23]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/"+i+"/"+k+'_'+"1_1test.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair50/"+i+"/"+k+'_'+j+'.ckpt'
            result_path="./result/pair50/unseen/test/"+i+"/"+k+'_'+j
            train_main(testfile_path,modelfile_path, result_path) 

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/"+i+"/"+k+'_'+"1_1test.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair50/"+i+"/"+k+'_'+j+'.ckpt'
            result_path="./result/pair50/unseen/test/"+i+"/"+k+'_'+j
            train_main(testfile_path,modelfile_path,result_path) 
            

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
1602 1936 692 2846
accuracy   0.6286
auc        0.7239
f1macro    0.6168
f1micro    0.6286
loss       24125.3958
precision0 0.6983
precision1 0.5951
recall0    0.4528
recall1    0.8044



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
1992 1546 936 2602
accuracy   0.6492
auc        0.7318
f1macro    0.6466
f1micro    0.6492
loss       20056.2897
precision0 0.6803
precision1 0.6273
recall0    0.5630
recall1    0.7354



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


...

[PREDICT] ----------------
382 3156 380 3158
accuracy   0.5003
auc        0.5054
f1macro    0.4094
f1micro    0.5003
loss       17686.7191
precision0 0.5013
precision1 0.5002
recall0    0.1080
recall1    0.8926



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
155 3383 136 3402
accuracy   0.5027
auc        0.5000
f1macro    0.3700
f1micro    0.5027
loss       41368.2237
precision0 0.5326
precision1 0.5014
recall0    0.0438
recall1    0.9616



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


[PREDICT] ----------------
97 3441 95 3443
accuracy   0.5003
auc        0.5115
f1macro    0.3564
f1micro    0.5003
loss       54343.8509
precision0 0.5052
precision1 0.5001
recall0    0.0274
recall1    0.9731



# unknown-validaiton

In [6]:
database=['healthy','patient']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4','1_6','1_8']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/validation/"+i+"/"+k+'_'+"1_1validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair50/"+i+"/"+k+'_'+j+'.ckpt'
            result_path="./result/pair50/unseen/validation/"+i+"/"+k+'_'+j
            train_main(testfile_path,modelfile_path, result_path) 

database=['Antigen_specificity']
name1=['1','2','3','4','5']
name=['1_1','1_2','1_4']
for i in database:
    for j in name:
        for k in name1:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/unknow/validation/"+i+"/"+k+'_'+"1_1validation.csv"
            modelfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/model/Retrain/pair50/"+i+"/"+k+'_'+j+'.ckpt'
            result_path="./result/pair50/unseen/validation/"+i+"/"+k+'_'+j
            train_main(testfile_path,modelfile_path,result_path) 
            

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


RangeIndex(start=0, stop=2568, step=1)
[PREDICT] ----------------
626 658 264 1020
accuracy   0.6410
auc        0.7149
f1macro    0.6323
f1micro    0.6410
loss       8316.2555
precision0 0.7034
precision1 0.6079
recall0    0.4875
recall1    0.7944



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


RangeIndex(start=0, stop=2568, step=1)
[PREDICT] ----------------
722 562 384 900
accuracy   0.6316
auc        0.7011
f1macro    0.6298
f1micro    0.6316
loss       7675.4947
precision0 0.6528
precision1 0.6156
recall0    0.5623
recall1    0.7009



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


...

RangeIndex(start=0, stop=2568, step=1)
[PREDICT] ----------------
199 1085 189 1095
accuracy   0.5039
auc        0.4955
f1macro    0.4351
f1micro    0.5039
loss       6072.7757
precision0 0.5129
precision1 0.5023
recall0    0.1550
recall1    0.8528



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


RangeIndex(start=0, stop=2568, step=1)
[PREDICT] ----------------
59 1225 68 1216
accuracy   0.4965
auc        0.5063
f1macro    0.3683
f1micro    0.4965
loss       12882.3193
precision0 0.4646
precision1 0.4982
recall0    0.0460
recall1    0.9470



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


RangeIndex(start=0, stop=2568, step=1)
[PREDICT] ----------------
38 1246 37 1247
accuracy   0.5004
auc        0.5186
f1macro    0.3581
f1micro    0.5004
loss       18774.7345
precision0 0.5067
precision1 0.5002
recall0    0.0296
recall1    0.9712



# TCR numbers top5(Parameter Adjustment)

In [33]:
import os
import sys
import csv
import time
from collections import deque
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import precision_recall_curve, auc
from data_loader import define_dataloader, load_embedding
from utils import get_performance_batchiter, print_performance, write_blackbox_output_batchiter
import data_io_tf

class Args:
    indepfile = None 
    blosum = None
    batch_size = 32
    #epoch = 50
    min_epoch = 1
    early_stop = True 
    lr = 0.001 
    cuda = True 
    seed = 1039 
    mode = 'train'
    save_model = True 
    model = 'attention'
    drop_rate = 0.25        
    lin_size = 1024 
    padding = "mid"
    heads = 5
    max_len_tcr = 20 
    max_len_pep = 22
    n_fold = 5 
    idx_test_fold = 0
    idx_val_fold = -1

def train_main(trainfile_path, testfile_path, result_path,epoch):
    import pandas as pd
    if torch.cuda.is_available():
        print("gpu enabled")
    else:
        print("no gpu")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    PRINT_EVERY_EPOCH = 1

    def train(model, device, train_loader, optimizer, epoch):
        model.train()
        for batch in train_loader:
            x_pep, x_tcr, y = batch.X_pep.to(device), batch.X_tcr.to(device), batch.y.to(device)
            optimizer.zero_grad()
            yhat = model(x_pep, x_tcr)
            y = y.unsqueeze(-1).expand_as(yhat)
            loss = F.binary_cross_entropy(yhat, y)
            loss.backward()
            optimizer.step()

        if epoch % PRINT_EVERY_EPOCH == 1:
            print('[TRAIN] Epoch {} Loss {:.4f}'.format(epoch, loss.item()))

    def evaluate_model(model, device, test_loader):
        model.eval()
        y_true = []
        y_scores = []
        with torch.no_grad():
            for batch in test_loader:
                x_pep, x_tcr, y = batch.X_pep.to(device), batch.X_tcr.to(device), batch.y.to(device)
                yhat = model(x_pep, x_tcr)
                y_true.extend(y.cpu().numpy())
                y_scores.extend(yhat.cpu().numpy())
        
        precision, recall, _ = precision_recall_curve(y_true, y_scores)
        auprc = auc(recall, precision)
        return auprc

    args = Args()

    device = torch.device('cuda' if args.cuda else 'cpu')

    torch.manual_seed(args.seed)
    if args.cuda:
        torch.cuda.manual_seed(args.seed)

    embedding_matrix = load_embedding(args.blosum)

    trainfile = pd.read_csv(trainfile_path)
    trainfile = trainfile.reset_index().rename(columns={'index': 'idx'})
    testfile = pd.read_csv(testfile_path)
    testfile['idx'] = testfile.index + len(trainfile)
    alldata = pd.concat([trainfile, testfile], axis=0)
    idx_train = trainfile['idx'].tolist()
    idx_test = testfile['idx'].tolist()
    x_pep = alldata['Epitope'].values
    x_tcr = alldata['CDR3B'].values
    y = alldata['Affinity'].values

    train_loader = define_dataloader(x_pep[idx_train], x_tcr[idx_train], y[idx_train],
                                     args.max_len_pep, args.max_len_tcr,
                                     padding=args.padding,
                                     batch_size=args.batch_size, device=device)
    test_loader = define_dataloader(x_pep[idx_test], x_tcr[idx_test], y[idx_test],
                                    maxlen_pep=train_loader['pep_length'],
                                    maxlen_tcr=train_loader['tcr_length'],
                                    padding=args.padding,
                                    batch_size=args.batch_size, device=device)

    if args.model == 'attention':
        from attention import Net
    else:
        raise ValueError('unknown model name')

    model = Net(embedding_matrix, args).to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    if args.mode == 'train':
        t0 = time.time()
        lossArraySize = 10
        lossArray = deque([sys.maxsize], maxlen=lossArraySize)
        for epoch in range(1, epoch + 1):
            train(model, device, train_loader['loader'], optimizer, epoch)
            perf_test = get_performance_batchiter(test_loader['loader'], model, device)
            lossArray.append(perf_test['loss'])
            average_loss_change = sum(np.abs(np.diff(lossArray))) / lossArraySize
            auprc = evaluate_model(model, device, test_loader['loader'])
            print(f'[EPOCH {epoch}] AUPRC: {auprc:.4f}')

            if epoch > args.min_epoch and average_loss_change < 10 and args.early_stop:
                break
        
        print('[TEST ] {} ----------------'.format(epoch))
        perf_test = get_performance_batchiter(test_loader['loader'], model, device)
        print_performance(perf_test)

        if args.save_model:
            wf_open1 = open(result_path + 'probability.csv', 'w', newline='')
            wf1 = csv.writer(wf_open1, delimiter=',')
            wf1.writerow(['Epitope', 'CDR3B', 'y_true', 'y_pred', 'y_prob'])
            write_blackbox_output_batchiter(test_loader, model, wf1, device, ifscore=True)
            wf_open1.close()

In [ ]:
reap=['reap1','reap2','reap3','reap4','reap5']
name=['top1','top2','top3','top4','top5']
number= ['3000', '2800', '2600', '2400', '2200', '2000', '1800', '1600', '1400', '1200', '1000', '800', '600', '400', '200', '50']
epochs=[50,70,90,100]
for i in reap:
    for j in name:
        for k in number:
            for epoch  in epochs:
                trainfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+k+'.csv'               
                testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/top5/healthy/"+i+"_" +j+"_"+'test.csv'             
                result_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/result/top5/seen/healthy/"+i+"_" +j+"_"+k+"_"+  str(epoch)
                train_main(trainfile_path, testfile_path, result_path,epoch)